# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import bibtexparser

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [2]:
parser = bibtexparser.bparser.BibTexParser(common_strings=True, ignore_nonstandard_types=False)
with open("../bibliography.bib") as bibtex_file:
    publications = bibtexparser.load(bibtex_file, parser=parser)

publications.entries

[{'author': 'Behbahani, Alireza R. and Von Moll, Alex and Zeller, Robert and Ordo, James',
  'date': '2014-09',
  'publisher': '{SAE International}',
  'booktitle': '2014 {{SAE Aerospace Systems}} and {{Technology Conference}}',
  'eventtitle': '2014 {{SAE Aerospace Systems}} and {{Technology Conference}}',
  'abstract': 'Modern propulsion system designers face challenges that require that aircraft and engine manufacturers improve performance as well as reduce the life-cycle cost (LCC). These improvements will require a more efficient, more reliable, and more advanced propulsion system. The concept of smart components is built around actively controlling the engine and the aircraft to operate optimally. Usage of smart components intelligently increases efficiency and system safety throughout the flight envelope, all while meeting environmental challenges. This approach requires an integration and optimization, both at the local level and the system level, to reduce cost. Interactions b

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [3]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [4]:
def parse_author(a):
    """Parses author name in the format of `Last, First Middle` where
    the middle name is sometimes there and sometimes not (and could just be an initial)
    
    Returns: author name as `F. M. Last`
    """
    
    a = a.split(', ')
    last = a[0].strip()
    fm = a[1].split(' ')
    first = fm[0][0] + '.'
    
    if len(fm) > 1:
        middle = fm[1][0] + '.'
    else:
        middle = ''
#     print([first, middle, last])
    
    if not middle == '':
        return first + ' ' + middle + ' ' + last
    else:
        return first + ' ' + last

# for row, item in publications.iterrows():
#     authors = ', '.join([parse_author(a) for a in item['Author'].split('; ')])

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
p = publications.entries[-1]
p = bibtexparser.customization.add_plaintext_fields(p)
p

{'note': 'Submitted for review',
 'author': 'Von Moll, Alexander and Casbeer, David and Garcia, Eloy and Milutinović, Dejan and Pachter, Meir',
 'date': '2019',
 'shortjournal': 'JINT',
 'journaltitle': 'Jounal of Intelligent \\& Robotic Systems',
 'abstract': "We consider a general pursuit-evasion differential game with three or more pursuers and a single evader, all with simple motion. It is shown that traditional means of differential game analysis is difficult for this scenario. But simple motion and min-max time to capture plus the two-person extension to Pontryagin's maximum principle imply straight-line motion at maximum speed which forms the basis of the solution using a geometric approach. Safe evader paths and policies are defined which guarantee the evader can reach its destination without getting captured by any of the pursuers, provided its destination satisfies some constraints. A linear program is used to characterize the solution and subsequently the saddle-point is com

In [9]:
import os


for item in publications.entries:
    
    item = bibtexparser.customization.add_plaintext_fields(item)
    year = item['plain_year'] if 'plain_year' in item else item['plain_date'].split('-')[0]
    key = item['plain_author'].split(',')[0].replace(' ', '').lower() + str(year) + item['plain_title'].split(' ')[0].lower()

    md_filename = key + ".md"
    html_filename = key
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item['plain_title'] + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
#     month = month_dict[item['plain_month']] if 'plain_month' in item else 6
#     day = item['plain_day'] if 'plain_day' in item else 15
#     date = datetime.date(int(year), int(month), int(day)).isoformat() + "00:00:00 + 0500"
    if not 'plain_date' in item:
        raise Exception(item)
    date = item['plain_date']
    if len(date) == 4:
        date += "-06-15 00:00:00 +0500"
    elif len(date) == 7:
        date += "-15 00:00:00 +0500"
    elif len(date) == 10:
        date += " 00:00:00 +0500"
    else:
        print(date)
        break
    
    
    md += "\ndate: " + date

    if 'plain_eventtitle' in item:
        venue = item['plain_eventtitle']
    elif 'plain_booktitle' in item:
        venue = item['plain_booktitle']
    elif 'plain_journaltitle' in item:
        venue = item['plain_journaltitle']
    elif 'plain_school' in item:
        venue = item['plain_institution']
    else:
        venue = False
    
    if 'plain_note' in item:
        note = item['plain_note']
        if 'submitted' in note.lower() or 'review' in note.lower() or 'accepted' in note.lower():
            venue += " (<b><i>" + note + "</i></b>)"
        
        
    if venue:
        md += "\nvenue: '" + html_escape(venue) + "'"
    
    if 'plain_url' in item:
        md += "\npaperurl: '" + item['plain_url'] + "'"
    
    if 'plain_doi' in item:
        md += "\ndoi: '" + item['plain_doi'] + "'"
        
    pubtypes = {"inproceedings": "conference",
                "article": "journal",
                "thesis": "academic"}
    md += "\npubtype: '" + pubtypes[item['ENTRYTYPE']] + "'"
    
#     md += "\ncitation: '" + html_escape(item.citation) + "'"
    authors = ', '.join([parse_author(a) for a in item['plain_author'].split(' and ')])
    md += "\nauthors: '" + authors + "'"
    
    md += "\nexcerpt_separator: \"\""
    
    md += "\n---"
    
    ## Markdown description for individual page
        
#     if len(str(item.excerpt)) > 5:
#         md += "\n" + html_escape(item.excerpt) + "\n"
    
    if 'plain_abstract' in item:
        md += "\n" + html_escape(item['plain_abstract']) + "\n"
    
    if 'plain_url' in item:
        md += "\n[Download paper here](" + item['plain_url'] + ")"
    
    if 'plain_doi' in item:
        md += "\n[DOI](" + item['plain_doi'] + ")"
        
#     md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
!ls ../_publications/

behbahani2014aircraft.md      vonmoll2014a.md
black2018integrated.md        vonmoll2014recent.md
kalyanam2018scalable.md       vonmoll2015machine.md
manyam2018shortest.md         vonmoll2018genetic.md
manyam2019coordinating.md     vonmoll2018pursuit-evasion.md
moslehi2018high-bandwidth.md  vonmoll2019multiple.md
vonmoll2013comparison.md


In [8]:
!cat ../_publications/behbahani2014aircraft.md

---
title: "Aircraft Integration Challenges and Opportunities for Distributed Intelligent Control, Power, Thermal Management, Diagnostic and Prognostic Systems"
collection: publications
permalink: /publication/behbahani2014aircraft
date: 2014-09-30 00:00:00 +0500
venue: 'SAE 2014 Aerospace Systems and Technology Conference'
paperurl: 'https://doi.org/10.4271/2014-01-2161'
authors: 'A. R. Behbahani, A. Von Moll, R. Zeller, J. Ordo'
excerpt_separator: ""
---
Modern propulsion system designers face challenges that require that aircraft and engine manufacturers improve performance as well as reduce the life-cycle cost (LCC). These improvements will require a more efficient, more reliable, and more advanced propulsion system. The concept of smart components is built around actively controlling the engine and the aircraft to operate optimally. Usage of smart components intelligently increases efficiency and system safety throughout the flight envelope, all while meeting environmental challen